<a href="https://colab.research.google.com/github/carlosgsouza/continuous-rag-eval-tutorial/blob/master/eval/data/Create_eval_dataset_from_HotpotQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -qU datasets rich

from rich import print

In [5]:
from datasets import load_dataset

hotpot_qa = load_dataset("hotpotqa/hotpot_qa", "distractor", trust_remote_code=True)
print(hotpot_qa)

qa_dataset = hotpot_qa['train'].filter(lambda qa: qa['level'] == 'medium').to_list()
print(len(qa_dataset))

print(qa_dataset[:1])

README.md:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

hotpot_qa.py:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 90447
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
})

Filter:   0%|          | 0/90447 [00:00<?, ? examples/s]

56814

[
    {
        'id': '5a7a06935542990198eaf050',
        'question': "Which magazine was started first Arthur's Magazine or First for Women?",
        'answer': "Arthur's Magazine",
        'type': 'comparison',
        'level': 'medium',
        'supporting_facts': {'title': ["Arthur's Magazine", 'First for Women'], 'sent_id': [0, 0]},
        'context': {
            'title': [
                'Radio City (Indian radio station)',
                'History of Albanian football',
                'Echosmith',
                "Women's colleges in the Southern United States",
                'First Arthur County Courthouse and Jail',
                "Arthur's Magazine",
                '2014–15 Ukrainian Hockey Championship',
                'First for Women',
                'Freeway Complex Fire',
                'William Rast'
            ],
            'sentences': [
                [
                    "Radio City is India's first private FM radio station and was started on 3 July 2001.",
                    ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was 
started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
                    ' It plays Hindi, English and regional songs.',
                    ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam 
October 2007.',
                    ' Radio City recently forayed into New Media in May 2008 with the launch of a music portal - 
PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.',
                    ' The Radio station currently plays a mix of Hindi and Regional music.',
                    ' Abraham Thomas is the CEO of the company.'
                ],
                [
                    'Football in Albania existed before the Albanian Football Federation (FSHF) was created.',
                    " This was evidenced by the team's registration at the Balkan Cup tournament during 1929-1931, 
which started in 1929 (although Albania eventually had pressure from the teams because of competition, competition 
started first and was strong enough in the duels) .",
                    ' Albanian National Team was founded on June 6, 1930, but Albania had to wait 16 years to play 
its first international match and then defeated Yugoslavia in 1946.',
                    ' In 1932, Albania joined FIFA (during the 12–16 June convention ) And in 1954 she was one of 
the founding members of UEFA.'
                ],
                [
                    'Echosmith is an American, Corporate indie pop band formed in February 2009 in Chino, 
California.',
                    ' Originally formed as a quartet of siblings, the band currently consists of Sydney, Noah and 
Graham Sierota, following the departure of eldest sibling Jamie in late 2016.',
                    ' Echosmith started first as "Ready Set Go!"',
                    ' until they signed to Warner Bros.',
                    ' Records in May 2012.',
                    ' They are best known for their hit song "Cool Kids", which reached number 13 on the 
"Billboard" Hot 100 and was certified double platinum by the RIAA with over 1,200,000 sales in the United States 
and also double platinum by ARIA in Australia.',
                    ' The song was Warner Bros.',
                    " Records' fifth-biggest-selling-digital song of 2014, with 1.3 million downloads sold.",
                    ' The band\'s debut album, "Talking Dreams", was released on October 8, 2013.'
                ],
                [
                    "Women's colleges in the Southern United States refers to undergraduate, bachelor's 
degree–granting institutions, often liberal arts colleges, whose student populations consist exclusively or almost 
exclusively of women, located in the Southern United States.",
                    " Many started first as girls' seminaries or academies.",
  

In [6]:
import json

def sanitize(text):
  s = ''.join(ch for ch in text if ord(ch) < 128)
  s = s.replace("'", "")
  s = s.replace("\"", "")
  return s


qas = qa_dataset[:100]

all_questions = []
for q in qas:

  context_docs = []
  for i, title in enumerate(q['context']['title']):
    doc = {
            "page_content": sanitize("\n".join(q['context']['sentences'][i])),
            "metadata": {
                "title": title,
                "question_id": q["id"],
            }
          }
    context_docs.append(doc)

  all_questions.append({
      "id": q["id"],
      "question": sanitize(q["question"]),
      "answer": sanitize(q["answer"]),
      "context": context_docs
  })

questions = all_questions[:80]
with open("questions.json", "w") as f:
  json.dump(questions, f, indent=2)

questions_without_context = []
for q in all_questions[80:90]:
  q["context"] = []
  q["answer"] = "I don't know"
  questions_without_context.append(q)
with open("questions_without_context.json", "w") as f:
  json.dump(questions_without_context, f, indent=2)

context_without_questions = []
for q in all_questions[90:]:
  del q["question"]
  del q["answer"]
  context_without_questions.append(q)
with open("context_without_questions.json", "w") as f:
  json.dump(context_without_questions, f, indent=2)

print(len(questions))
print(len(questions_without_context))
print(len(context_without_questions))

80

10

10